<a href="https://colab.research.google.com/github/Daksh-create349/My-Python-Case-study/blob/main/Final_137.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime

# Decorator to validate date format
def validate_date(func):
    def wrapper(self, emp_id, name, dob):
        try:
            datetime.strptime(dob, "%Y-%m-%d")
        except ValueError:
            print(" Invalid DOB format! Use YYYY-MM-DD")
            return
        return func(self, emp_id, name, dob)
    return wrapper


class Employee:
    def __init__(self, emp_id, name, dob):
        self.emp_id = emp_id
        self.name = name
        self.dob = dob

    def __repr__(self):
        return f"{self.emp_id} | {self.name} | {self.dob}"


In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os

def validate_date(func):
    def wrapper(self, emp_id, name, dob):
        try:
            datetime.strptime(dob, "%Y-%m-%d")
            return func(self, emp_id, name, dob)
        except ValueError:
            print("Error: Invalid Date Format! Please use YYYY-MM-DD.")
    return wrapper

class BirthdayReminder:

    def __init__(self, emp_file="employees (1).csv", log_file="reminders_log.csv"):
        self.emp_file = emp_file
        self.log_file = log_file

        if not os.path.exists(self.emp_file):
            pd.DataFrame(columns=["EmployeeID", "Name", "DOB"]).to_csv(self.emp_file, index=False)

        if not os.path.exists(self.log_file):
             pd.DataFrame(columns=["LogID", "EmployeeID", "Date", "Status"]).to_csv(self.log_file, index=False)

    @validate_date
    def add_employee(self, emp_id, name, dob):
        df = pd.read_csv(self.emp_file)

        if str(emp_id) in df["EmployeeID"].astype(str).values:
            print("Employee ID already exists!")
            return

        new_row = {"EmployeeID": emp_id, "Name": name, "DOB": dob}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(self.emp_file, index=False)
        print("Employee added successfully!")

    def search_by_month(self, month):
        df = pd.read_csv(self.emp_file)
        df["DOB_DT"] = pd.to_datetime(df["DOB"])

        current_year = datetime.now().year
        df["Age"] = df["DOB_DT"].apply(lambda x: current_year - x.year)

        result = df[df["DOB_DT"].dt.month == month]
        print(f"\n--- Birthdays in Month {month} ---")
        print(result[["EmployeeID", "Name", "DOB", "Age"]])

    def find_upcoming(self):
        df = pd.read_csv(self.emp_file)
        today = datetime.now().date()
        next_week = today + timedelta(days=7)

        def get_bday_this_year(dob_str):
            try:
                d = datetime.strptime(str(dob_str), "%Y-%m-%d").date()
                return d.replace(year=today.year)
            except ValueError:
                return d.replace(year=today.year, month=3, day=1)

        df["BirthdayThisYear"] = df["DOB"].apply(get_bday_this_year)

        result = df[(df["BirthdayThisYear"] >= today) & (df["BirthdayThisYear"] <= next_week)]

        print("\n--- Upcoming Birthdays (Next 7 Days) ---")
        if not result.empty:
            print(result[["EmployeeID", "Name", "DOB"]])
        else:
            print("No upcoming birthdays.")
        return result

    def send_reminders(self):
        upcoming = self.find_upcoming()
        if upcoming.empty: return

        log_df = pd.read_csv(self.log_file)
        for _, row in upcoming.iterrows():
            new_log = {
                "LogID": f"R{1000 + len(log_df)}",
                "EmployeeID": row["EmployeeID"],
                "Date": str(datetime.now().date()),
                "Status": "Sent"
            }
            log_df = pd.concat([log_df, pd.DataFrame([new_log])], ignore_index=True)
            print(f"Reminder sent to {row['Name']}")

        log_df.to_csv(self.log_file, index=False)
        print("Log Updated!")

    def export_calendar(self, month):
        df = pd.read_csv(self.emp_file)
        df["DOB_DT"] = pd.to_datetime(df["DOB"])
        monthly = df[df["DOB_DT"].dt.month == month]

        if not monthly.empty:
            filename = f"Calendar_Month_{month}.csv"
            monthly[["Name", "DOB"]].to_csv(filename, index=False)
            print(f"Calendar exported to {filename}")
        else:
            print("No birthdays in this month to export.")

    def show_chart(self):
        df = pd.read_csv(self.emp_file)
        df["Month"] = pd.to_datetime(df["DOB"]).dt.month
        counts = df["Month"].value_counts().sort_index().reindex(range(1, 13), fill_value=0)

        plt.figure(figsize=(10, 5))
        plt.bar(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], counts.values, color='purple')
        plt.title("Birthday Distribution")
        plt.show()

In [ ]:
from IPython.display import clear_output
import pandas as pd
import os

rem = BirthdayReminder(emp_file="employees (1).csv")

while True:
    print("\n--- Birthday Reminder System ---")
    print("1. Add Employee")
    print("2. Search by Month")
    print("3. Find Upcoming Birthdays (7 days)")
    print("4. Send Reminders")
    print("5. Export Monthly Calendar")
    print("6. Show Birthday Chart")
    print("7. Show employees (1).csv")
    print("8. Show reminders_log.csv")
    print("9. Exit")

    choice = input("\nEnter choice: ")

    clear_output()

    if choice == "1":
        emp_id = input("Employee ID: ")
        name = input("Name: ")
        dob = input("DOB (YYYY-MM-DD): ")
        rem.add_employee(emp_id, name, dob)

    elif choice == "2":
        try:
            m = int(input("Enter month (1–12): "))
            rem.search_by_month(m)
        except ValueError:
            print("Invalid number.")

    elif choice == "3":
        rem.find_upcoming()

    elif choice == "4":
        rem.send_reminders()

    elif choice == "5":
        try:
            m = int(input("Enter month to export (1–12): "))
            rem.export_calendar(m)
        except ValueError:
            print("Invalid number.")

    elif choice == "6":
        rem.show_chart()

    elif choice == "7":
        if os.path.exists("employees (1).csv"):
            print(pd.read_csv("employees (1).csv"))
        else:
            print("File 'employees (1).csv' not found.")

    elif choice == "8":
        if os.path.exists("reminders_log.csv"):
            print(pd.read_csv("reminders_log.csv"))

    elif choice == "9":
        print("Thank u for using ~Daksh Ranjan srivastava")
        break

    else:
        print("Invalid choice")

Thank u for using ~Daksh Ranjan srivastava
